In [1]:
# %pip install -qU transformers accelerate einops langchain xformers bitsandbytes
# %pip install scipy

In [1]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [2]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = 'meta-llama/Llama-2-70b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


hf_token = os.environ.get('hf_token')
# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Model loaded on cuda:0


In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [4]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "From the collector RRC06, gather BGP announcements for the IPv6 prefixes over a period from April 5, 2020, 12:00 to April 12, 2020, 12:00 and conduct a time-series analysis to observe trends. Use PyBGPStream for this task."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] From the collector RRC06, gather BGP announcements for the IPv6 prefixes over a period from April 5, 2020, 12:00 to April 12, 2020, 12:00 and conduct a time-series analysis to observe trends. Use PyBGPStream for this task. [/INST]  Sure, here's an example of how you could use PyBGPStream to gather BGP announcements for IPv6 prefixes and conduct a time-series analysis to observe trends:
```
import pybgpstream
import pandas as pd
import matplotlib.pyplot as plt

# Set up the BGP collector
collector = pybgpstream.BGPCollector(RRC06)

# Define the time period for the analysis
start_time = '2020-04-05 12:00:00'
end_time = '2020-04-12 12:00:00'

# Gather BGP announcements for IPv6 prefixes during the defined time period
announcements = collector.get_announcements(start_time, end_time, afi='ipv6')

# Convert the announcements to a pandas DataFrame
df = pd.DataFrame(announcements)

# Remove any duplicate announcements
df = df.drop_duplicates()

# Group the announcements by day and co

In [4]:
import json
input_file_path = "/home/hb/LLM-research/evaluation/bgp_test_QA_val.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]What is the purpose of BGP Update messages, and how do they carry route information?[/INST]  BGP (Border Gateway Protocol) Update messages are used to exchange routing information between BGP peers. The primary purpose of BGP Update messages is to inform other BGP peers of changes in the routing table, such as new routes, withdrawn routes, or changes in the path attributes of existing routes.

BGP Update messages carry route information in the form of BGP routes, which consist of a destination IP address, a next hop IP address, and path attributes such as the AS path, origin, and MRAI (Multi-Exit Routing Abstraction Interface). The BGP Update message includes a list of BGP routes, known as the "route list," which contains the updated routing information.

Each BGP route in the route list includes the following information:

1. Destination IP address: This is the IP address of the network that the route is associated with.
2. Next hop IP address: This is the IP address of 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Explain the different BGP neighbor states and the transitions between them.[/INST]  BGP (Border Gateway Protocol) is a routing protocol used by routers to exchange information about networks and reachability. BGP neighbors are the peering relationships between routers that exchange BGP messages. The neighbor states of BGP define the different phases of the relationship between two BGP peers.

There are six different BGP neighbor states, as defined in RFC 4271:

1. Idle: This is the initial state of a BGP neighbor relationship. In this state, the router has not yet sent any BGP messages to its peer.
2. Connect: In this state, the router is trying to establish a BGP connection with its peer. It sends a "connect" message to the peer and waits for a response.
3. Active: Once the connection is established, the router moves to the Active state. In this state, the router sends BGP updates to its peer and receives updates from its peer.
4. OpenSent: In this state, the router has 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Output: [INST]What are the steps for diagnosing BGP route problems, and how can these steps help in troubleshooting BGP issues?[/INST]  Diagnosing BGP route problems can be a challenging task, but following a structured approach can help identify and resolve issues quickly. Here are some steps that can help in troubleshooting BGP issues:

1. Gather Information:
	* Collect relevant information about the BGP setup, such as BGP configurations, routing tables, and network topology.
	* Review system logs and messages to identify any error messages or warnings related to BGP.
2. Identify the Problem:
	* Determine the nature of the problem, such as missing routes, incorrect routes, or route leaks.
	* Use tools like `tcpdump` or `Wireshark` to capture and analyze BGP updates and communication between BGP peers.
3. Isolate the Issue:
	* Use techniques like narrowing down the scope of the issue, such as disabling BGP peering sessions or filtering routes to isolate the problem.
	* Use tools like 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Can you provide an example of a large-scale service provider BGP implementation and its benefits?[/INST]  Sure, here's an example of a large-scale service provider BGP implementation and its benefits:

Example:

A large cloud service provider, let's call it "CloudX," operates a global network with hundreds of data centers and points of presence (PoPs) across different regions. CloudX offers various services, including compute, storage, and networking, to its customers. To provide efficient and scalable network connectivity between its data centers and customer sites, CloudX implements BGP in its network.

Benefits:

1. Scalability: BGP enables CloudX to scale its network to meet the growing demands of its customers. With BGP, CloudX can easily add new data centers and PoPs to its network, and automatically distribute routing information among all nodes. This allows CloudX to efficiently serve a large number of customers and handle increasing network traffic.
2. Flexibilit

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Could you share a notable BGP incident involving route hijacking and its impact on network operations?[/INST]  One notable BGP incident involving route hijacking occurred in 2017, when a Russian telecommunications company, Rostelecom, accidentally hijacked internet traffic destined for Google and other major internet services. The incident had a significant impact on network operations and highlighted the vulnerabilities of the Border Gateway Protocol (BGP) to route hijacking.

On October 12, 2017, Rostelecom, one of Russia's largest telecommunications companies, accidentally leaked a BGP update to its peers, claiming ownership of several large blocks of IP addresses, including those belonging to Google and other major internet services. This BGP update effectively hijacked internet traffic destined for these services, redirecting it through Rostelecom's network.

The impact of the incident was immediate and far-reaching. Internet users around the world began experiencing

In [ ]:
import json

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_multiple_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"{question}\nChoose the correct answer:\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
        # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Generated Answer: When analyzing BGP route propagation, which attribute provides information about the path that the BGP update has traversed?
Choose the correct answer:
a) AS-Path
b) MED (Multi-Exit Discriminator)
c) Local Preference
d) Next Hop

Answer: a) AS-Path

Explanation:
The AS-Path attribute in BGP provides information about the path that the BGP update has traversed. It is a list of autonomous system numbers (ASNs) that the update has passed through, with the originating ASN at
Parsed_answer: a
----------------1-----------------
Generated Answer: Which of the following is a primary purpose of BGP?
Choose the correct answer:
a) Establishing communication between devices within a LAN
b) Exchanging routing information between autonomous systems
c) Optimizing local network traffic
d) Assigning IP addresses to devices within a network

Answer: b) Exchanging routing information between autonomous systems

Explanation: BGP (Border Gateway Protocol) is a protocol used for exchanging

In [ ]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_fill_the_blank.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
filled_answers = []

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST]{question} \n Fill the blank: \n [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    # print(f"Output: {generated_answer}")
    
    generated_answer = re.search(r'\[/INST\]\s+(.*)', generated_answer, re.DOTALL).group(1).strip()
    if "Sure" in generated_answer:
        generated_answer = generated_answer.split('\n', 1)[1]  # Skip the first line
    print(f"Generated Answer: {generated_answer}")
    generated_answers.append(generated_answer)
    
    # Find the position of the placeholder in the first string
    placeholder_position = question.find("________")

    # Extract the content filled in the placeholder
    filled_content = generated_answer[placeholder_position:placeholder_position + len("autonomous systems (ASes)")]
    filled_answers.append(filled_content)
    n_question += 1
    print(f"Filled content: {filled_content}")
    print(f"---------------------{n_question}-----------------------------")


Generated Answer: Autonomous Systems (AS).

BGP is used to exchange routing and reachability information between different Autonomous Systems (AS) on the Internet. An Autonomous System is a group of networks that are managed by a single entity, such as an Internet Service Provider (ISP) or
Filled content: Internet. An Autonomous S
---------------------1-----------------------------
Generated Answer: In BGP, the AS-Path attribute represents the sequence of autonomous systems (ASes) that a route has traversed.
Filled content: sequence of autonomous sy
---------------------2-----------------------------
Generated Answer: BGP routers exchange routing information using UPDATE messages, which contain information about reachable destinations.
Filled content: UPDATE messages, which co
---------------------3-----------------------------
Generated Answer: BGP route aggregation is the process of combining multiple smaller IP prefixes into a single aggregated prefix.
Filled content: aggregation is